In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


import hydra
from typing import Optional
from collections.abc import Callable
from omegaconf import OmegaConf, DictConfig
from data_module.lightning_data_module import DataModule
from hydra import initialize, initialize_config_module, initialize_config_dir, compose

1. bevdepth data load O
2. our data load O
3. get item 비교 O
4. dataloader -> 다른순서사진나옴
4. 추후에 our dataset과 bevdepth dataset의 세세한 matrix 등도 전부 비교 필요 !! 
----------------
0. bev depth 로직 파악 O
1. BEVdepth's data 이용   
    -  dataset은 datamodule이용(config일단 여따 일단 넣고)) O
    - 기존 bevdepth와 동일한 데이터 가져오는지 확인 test  O
    - dataloader config들은 hydra이용. O
    - hydra 넣고 동일확인 test O
2. model cvt로 이전 작업   
## Dataset 값 비교

In [2]:
def setup(cfg):

    cfg.loader.batch_size = 1

    if 'split' not in cfg:
        cfg.split = 'val'

    if 'shuffle' not in cfg:
        cfg.shuffle = False
        
def setup_config(cfg: DictConfig, override: Optional[Callable] = None):

    OmegaConf.set_struct(cfg, False)
    OmegaConf.resolve(cfg)
    OmegaConf.set_struct(cfg, True)


def merge_rgbs(imgs):

    fl = np.transpose(imgs[0], (1,2,0))
    ff = np.transpose(imgs[1], (1,2,0))
    fr = np.transpose(imgs[2], (1,2,0))

    front = np.hstack([fl, ff, fr])

    bl = np.transpose(imgs[5], (1,2,0))
    bf = np.transpose(imgs[4], (1,2,0))
    br = np.transpose(imgs[3], (1,2,0))

    back = np.hstack([bl, bf, br])

    whole = np.vstack([front, back])

    return whole


In [3]:
#! Our dataloader
# # *  config setup  * #
CONFIG_PATH = '/usr/src/CV_For_Autonomous_Driving/config'

with initialize_config_dir(config_dir=CONFIG_PATH):
    cfg = compose(config_name='default_config_debug.yaml')

print(cfg.data)

{'img_conf': {'img_mean': [123.675, 116.28, 103.53], 'img_std': [58.395, 57.12, 57.375], 'to_rgb': True}, 'ida_aug_conf': {'resize_lim': [0.386, 0.55], 'final_dim': [256, 704], 'rot_lim': [-5.4, 5.4], 'H': 900, 'W': 1600, 'rand_flip': True, 'bot_pct_lim': [0.0, 0.0], 'cams': ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT'], 'Ncams': 6}, 'bda_aug_conf': {'rot_lim': [-22.5, 22.5], 'scale_lim': [0.95, 1.05], 'flip_dx_ratio': 0.5, 'flip_dy_ratio': 0.5}, 'data_root': '/usr/src/nuscenes', 'num_sweeps': 0, 'sweep_idxes': [], 'key_idxes': [], 'classes': ['car', 'truck', 'construction_vehicle', 'bus', 'trailer', 'barrier', 'motorcycle', 'bicycle', 'pedestrian', 'traffic_cone'], 'use_fusion': False}


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  """


In [4]:
#! Our dataloader
# # *  config setup  * #
CONFIG_PATH = '/usr/src/CV_For_Autonomous_Driving/config'

with initialize_config_dir(config_dir=CONFIG_PATH):
    cfg = compose(config_name='default_config_debug.yaml')

    
setup_config(cfg, setup)

# dataset list 만드는 test
DM = DataModule('nusc_det_dataset', cfg.data, cfg.loader)
# split = 'train'
# data_cfg_our = DM.data_cfg
# datasets = DM.get_data(split=split,
#                             **data_cfg_our)
train_dataloader = DM.train_dataloader()



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  """


True True True


In [5]:
#! BEVDepth dataloader
from model_module.bev_depth_lss_r50_256x704_128x128_24e_2key import BEVDepthLightningModel
model_module = BEVDepthLightningModel()


2023-01-15 15:10:34,139 - mmcv - INFO - initialize SECONDFPN with init_cfg [{'type': 'Kaiming', 'layer': 'ConvTranspose2d'}, {'type': 'Constant', 'layer': 'NaiveSyncBatchNorm2d', 'val': 1.0}]
2023-01-15 15:10:34,151 - mmcv - INFO - 
deblocks.0.0.weight - torch.Size([128, 256, 4, 4]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2023-01-15 15:10:34,152 - mmcv - INFO - 
deblocks.0.1.weight - torch.Size([128]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2023-01-15 15:10:34,153 - mmcv - INFO - 
deblocks.0.1.bias - torch.Size([128]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2023-01-15 15:10:34,154 - mmcv - INFO - 
deblocks.1.0.weight - torch.Size([128, 512, 2, 2]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2023-01-15 15:10:34,155 - mmcv - INFO - 
deblocks.1.1.weight - torch.Size([128]): 
The value is the same before and after calling `init_weig

In [6]:
train = iter(train_dataloader)
train_bd= iter(model_module.train_dataloader())

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

In [7]:

data = next(train)
data_bd = next(train_bd)

### 동일한 Data인지 비교

In [8]:
import torch
import numpy as np
print('0.imgs : ', [torch.all(data[0][i]) == torch.all(data_bd[0][i]) for i in range(len(data[0])) ])
print('0.imgs : ',torch.all(data[0]) == torch.all(data_bd[0]))
print('1.mats_dict : ',[np.all(np.asarray(data[1][i])) == np.all(np.asarray(data_bd[1][i])) \
                for i in data[1].keys()])
print('2.timestamps : ',torch.all(data[2]) == torch.all(data_bd[2]))
data3 =[np.asarray(data[3][i][k])== np.asarray(data_bd[3][i][k]) for i in range(len(data[3])) for k in data[3][i].keys() ]
print('3.img_metas : ',*data3, sep='\n')
print('4.gt_boxes : ', [torch.all(data[4][i]) == torch.all(data_bd[4][i]) for i in range(len(data[4])) ])
print('5.gt_labaels : ', [torch.all(data[5][i]) == torch.all(data_bd[5][i]) for i in range(len(data[5])) ])

0.imgs :  [tensor(True), tensor(True), tensor(True), tensor(True)]
0.imgs :  tensor(True)
1.mats_dict :  [True, True, True, True, True]
2.timestamps :  tensor(True)
3.img_metas : 
True
[ True  True  True]
[ True  True  True  True]
True
True
[ True  True  True]
[ True  True  True  True]
True
True
[ True  True  True]
[ True  True  True  True]
True
True
[ True  True  True]
[ True  True  True  True]
True
4.gt_boxes :  [tensor(True), tensor(True), tensor(True), tensor(True)]
5.gt_labaels :  [tensor(True), tensor(True), tensor(True), tensor(True)]


In [9]:
len(data_bd[4][0])

10

In [10]:
len(data[4][0])

10

In [11]:
cfg.loader

{'batch_size': 4, 'num_workers': 8, 'pin_memory': True, 'prefetch_factor': 8}

In [12]:
import torch
import numpy as np
print('0.imgs : ', [torch.all(past_bd[0][i]) == torch.all(data_bd[0][i]) for i in range(len(past_bd[0])) ])
print('0.imgs : ',torch.all(data[0]) == torch.all(past[0]))
print('1.mats_dict : ',[np.all(np.asarray(data[1][i])) == np.all(np.asarray(data_bd[1][i])) \
                for i in data[1].keys()])
print('2.timestamps : ',torch.all(data[2]) == torch.all(past[2]))
data3 =[np.asarray(data_bd[3][i][k])== np.asarray(past[3][i][k]) for i in range(len(data[3])) for k in data[3][i].keys() ]
print('3.img_metas : ',*data3, sep='\n')
print('4.gt_boxes : ', [torch.all(data[4][i]) == torch.all(data_bd[4][i]) for i in range(len(data[4])) ])
print('5.gt_labaels : ', [torch.all(data[5][i]) == torch.all(data_bd[5][i]) for i in range(len(data[5])) ])

NameError: name 'past_bd' is not defined

In [ ]:
past_bd = data_bd
past = data

In [14]:
len(train_dataloader.dataset)

28130

In [16]:
H = 900
W = 1600
final_dim = (256, 704)
backbone_conf = {
    'x_bound': [-51.2, 51.2, 0.8],
    'y_bound': [-51.2, 51.2, 0.8],
    'z_bound': [-5, 3, 8],
    'd_bound': [2.0, 58.0, 0.5],
    'final_dim':
    final_dim,
    'output_channels':
    80,
    'downsample_factor':
    16,
    'img_backbone_conf':
    dict(
        type='ResNet',
        depth=50,
        frozen_stages=0,
        out_indices=[0, 1, 2, 3],
        norm_eval=False,
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50'),
    ),
    'img_neck_conf':
    dict(
        type='SECONDFPN',
        in_channels=[256, 512, 1024, 2048],
        upsample_strides=[0.25, 0.5, 1, 2],
        out_channels=[128, 128, 128, 128],
    ),
    'depth_net_conf':
    dict(in_channels=512, mid_channels=512)
}


bev_backbone = dict(
    type='ResNet',
    in_channels=80,
    depth=18,
    num_stages=3,
    strides=(1, 2, 2),
    dilations=(1, 1, 1),
    out_indices=[0, 1, 2],
    norm_eval=False,
    base_channels=160,
)

bev_neck = dict(type='SECONDFPN',
                in_channels=[80, 160, 320, 640],
                upsample_strides=[1, 2, 4, 8],
                out_channels=[64, 64, 64, 64])

CLASSES = [
    'car',
    'truck',
    'construction_vehicle',
    'bus',
    'trailer',
    'barrier',
    'motorcycle',
    'bicycle',
    'pedestrian',
    'traffic_cone',
]

TASKS = [
    dict(num_class=1, class_names=['car']),
    dict(num_class=2, class_names=['truck', 'construction_vehicle']),
    dict(num_class=2, class_names=['bus', 'trailer']),
    dict(num_class=1, class_names=['barrier']),
    dict(num_class=2, class_names=['motorcycle', 'bicycle']),
    dict(num_class=2, class_names=['pedestrian', 'traffic_cone']),
]

common_heads = dict(reg=(2, 2),
                    height=(1, 2),
                    dim=(3, 2),
                    rot=(2, 2),
                    vel=(2, 2))

bbox_coder = dict(
    type='CenterPointBBoxCoder',
    post_center_range=[-61.2, -61.2, -10.0, 61.2, 61.2, 10.0],
    max_num=500,
    score_threshold=0.1,
    out_size_factor=4,
    voxel_size=[0.2, 0.2, 8],
    pc_range=[-51.2, -51.2, -5, 51.2, 51.2, 3],
    code_size=9,
)

train_cfg = dict(
    point_cloud_range=[-51.2, -51.2, -5, 51.2, 51.2, 3],
    grid_size=[512, 512, 1],
    voxel_size=[0.2, 0.2, 8],
    out_size_factor=4,
    dense_reg=1,
    gaussian_overlap=0.1,
    max_objs=500,
    min_radius=2,
    code_weights=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5],
)

test_cfg = dict(
    post_center_limit_range=[-61.2, -61.2, -10.0, 61.2, 61.2, 10.0],
    max_per_img=500,
    max_pool_nms=False,
    min_radius=[4, 12, 10, 1, 0.85, 0.175],
    score_threshold=0.1,
    out_size_factor=4,
    voxel_size=[0.2, 0.2, 8],
    nms_type='circle',
    pre_max_size=1000,
    post_max_size=83,
    nms_thr=0.2,
)

head_conf = {
    'bev_backbone_conf': bev_backbone,
    'bev_neck_conf': bev_neck,
    'tasks': TASKS,
    'common_heads': common_heads,
    'bbox_coder': bbox_coder,
    'train_cfg': train_cfg,
    'test_cfg': test_cfg,
    'in_channels': 256,  # Equal to bev_neck output_channels.
    'loss_cls': dict(type='GaussianFocalLoss', reduction='mean'),
    'loss_bbox': dict(type='L1Loss', reduction='mean', loss_weight=0.25),
    'gaussian_overlap': 0.1,
    'min_radius': 2,
}



backbone_conf = {
    'x_bound': [-51.2, 51.2, 0.8],
    'y_bound': [-51.2, 51.2, 0.8],
    'z_bound': [-5, 3, 8],
    'd_bound': [2.0, 58.0, 0.5],
    'final_dim':
    final_dim,
    'output_channels':
    80,
    'downsample_factor':
    16,
    'img_backbone_conf':
    dict(
        type='ResNet',
        depth=50,
        frozen_stages=0,
        out_indices=[0, 1, 2, 3],
        norm_eval=False,
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50'),
    ),
    'img_neck_conf':
    dict(
        type='SECONDFPN',
        in_channels=[256, 512, 1024, 2048],
        upsample_strides=[0.25, 0.5, 1, 2],
        out_channels=[128, 128, 128, 128],
    ),
    'depth_net_conf':
    dict(in_channels=512, mid_channels=512)
}


bev_backbone = dict(
    type='ResNet',
    in_channels=80,
    depth=18,
    num_stages=3,
    strides=(1, 2, 2),
    dilations=(1, 1, 1),
    out_indices=[0, 1, 2],
    norm_eval=False,
    base_channels=160,
)

bev_neck = dict(type='SECONDFPN',
                in_channels=[80, 160, 320, 640],
                upsample_strides=[1, 2, 4, 8],
                out_channels=[64, 64, 64, 64])

CLASSES = [
    'car',
    'truck',
    'construction_vehicle',
    'bus',
    'trailer',
    'barrier',
    'motorcycle',
    'bicycle',
    'pedestrian',
    'traffic_cone',
]

TASKS = [
    dict(num_class=1, class_names=['car']),
    dict(num_class=2, class_names=['truck', 'construction_vehicle']),
    dict(num_class=2, class_names=['bus', 'trailer']),
    dict(num_class=1, class_names=['barrier']),
    dict(num_class=2, class_names=['motorcycle', 'bicycle']),
    dict(num_class=2, class_names=['pedestrian', 'traffic_cone']),
]

common_heads = dict(reg=(2, 2),
                    height=(1, 2),
                    dim=(3, 2),
                    rot=(2, 2),
                    vel=(2, 2))

bbox_coder = dict(
    type='CenterPointBBoxCoder',
    post_center_range=[-61.2, -61.2, -10.0, 61.2, 61.2, 10.0],
    max_num=500,
    score_threshold=0.1,
    out_size_factor=4,
    voxel_size=[0.2, 0.2, 8],
    pc_range=[-51.2, -51.2, -5, 51.2, 51.2, 3],
    code_size=9,
)

train_cfg = dict(
    point_cloud_range=[-51.2, -51.2, -5, 51.2, 51.2, 3],
    grid_size=[512, 512, 1],
    voxel_size=[0.2, 0.2, 8],
    out_size_factor=4,
    dense_reg=1,
    gaussian_overlap=0.1,
    max_objs=500,
    min_radius=2,
    code_weights=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5],
)

test_cfg = dict(
    post_center_limit_range=[-61.2, -61.2, -10.0, 61.2, 61.2, 10.0],
    max_per_img=500,
    max_pool_nms=False,
    min_radius=[4, 12, 10, 1, 0.85, 0.175],
    score_threshold=0.1,
    out_size_factor=4,
    voxel_size=[0.2, 0.2, 8],
    nms_type='circle',
    pre_max_size=1000,
    post_max_size=83,
    nms_thr=0.2,
)

head_conf = {
    'bev_backbone_conf': bev_backbone,
    'bev_neck_conf': bev_neck,
    'tasks': TASKS,
    'common_heads': common_heads,
    'bbox_coder': bbox_coder,
    'train_cfg': train_cfg,
    'test_cfg': test_cfg,
    'in_channels': 256,  # Equal to bev_neck output_channels.
    'loss_cls': dict(type='GaussianFocalLoss', reduction='mean'),
    'loss_bbox': dict(type='L1Loss', reduction='mean', loss_weight=0.25),
    'gaussian_overlap': 0.1,
    'min_radius': 2,
}

In [17]:
backbone_conf

{'x_bound': [-51.2, 51.2, 0.8],
 'y_bound': [-51.2, 51.2, 0.8],
 'z_bound': [-5, 3, 8],
 'd_bound': [2.0, 58.0, 0.5],
 'final_dim': (256, 704),
 'output_channels': 80,
 'downsample_factor': 16,
 'img_backbone_conf': {'type': 'ResNet',
  'depth': 50,
  'frozen_stages': 0,
  'out_indices': [0, 1, 2, 3],
  'norm_eval': False,
  'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}},
 'img_neck_conf': {'type': 'SECONDFPN',
  'in_channels': [256, 512, 1024, 2048],
  'upsample_strides': [0.25, 0.5, 1, 2],
  'out_channels': [128, 128, 128, 128]},
 'depth_net_conf': {'in_channels': 512, 'mid_channels': 512}}

In [18]:
bev_backbone

{'type': 'ResNet',
 'in_channels': 80,
 'depth': 18,
 'num_stages': 3,
 'strides': (1, 2, 2),
 'dilations': (1, 1, 1),
 'out_indices': [0, 1, 2],
 'norm_eval': False,
 'base_channels': 160}

In [19]:
bev_neck

{'type': 'SECONDFPN',
 'in_channels': [80, 160, 320, 640],
 'upsample_strides': [1, 2, 4, 8],
 'out_channels': [64, 64, 64, 64]}

In [20]:
TASKS

[{'num_class': 1, 'class_names': ['car']},
 {'num_class': 2, 'class_names': ['truck', 'construction_vehicle']},
 {'num_class': 2, 'class_names': ['bus', 'trailer']},
 {'num_class': 1, 'class_names': ['barrier']},
 {'num_class': 2, 'class_names': ['motorcycle', 'bicycle']},
 {'num_class': 2, 'class_names': ['pedestrian', 'traffic_cone']}]

In [21]:
common_heads

{'reg': (2, 2), 'height': (1, 2), 'dim': (3, 2), 'rot': (2, 2), 'vel': (2, 2)}

In [22]:
bbox_coder

{'type': 'CenterPointBBoxCoder',
 'post_center_range': [-61.2, -61.2, -10.0, 61.2, 61.2, 10.0],
 'max_num': 500,
 'score_threshold': 0.1,
 'out_size_factor': 4,
 'voxel_size': [0.2, 0.2, 8],
 'pc_range': [-51.2, -51.2, -5, 51.2, 51.2, 3],
 'code_size': 9}

In [23]:
train_cfg

{'point_cloud_range': [-51.2, -51.2, -5, 51.2, 51.2, 3],
 'grid_size': [512, 512, 1],
 'voxel_size': [0.2, 0.2, 8],
 'out_size_factor': 4,
 'dense_reg': 1,
 'gaussian_overlap': 0.1,
 'max_objs': 500,
 'min_radius': 2,
 'code_weights': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5]}

In [24]:
test_cfg

{'post_center_limit_range': [-61.2, -61.2, -10.0, 61.2, 61.2, 10.0],
 'max_per_img': 500,
 'max_pool_nms': False,
 'min_radius': [4, 12, 10, 1, 0.85, 0.175],
 'score_threshold': 0.1,
 'out_size_factor': 4,
 'voxel_size': [0.2, 0.2, 8],
 'nms_type': 'circle',
 'pre_max_size': 1000,
 'post_max_size': 83,
 'nms_thr': 0.2}

In [25]:
head_conf

{'bev_backbone_conf': {'type': 'ResNet',
  'in_channels': 80,
  'depth': 18,
  'num_stages': 3,
  'strides': (1, 2, 2),
  'dilations': (1, 1, 1),
  'out_indices': [0, 1, 2],
  'norm_eval': False,
  'base_channels': 160},
 'bev_neck_conf': {'type': 'SECONDFPN',
  'in_channels': [80, 160, 320, 640],
  'upsample_strides': [1, 2, 4, 8],
  'out_channels': [64, 64, 64, 64]},
 'tasks': [{'num_class': 1, 'class_names': ['car']},
  {'num_class': 2, 'class_names': ['truck', 'construction_vehicle']},
  {'num_class': 2, 'class_names': ['bus', 'trailer']},
  {'num_class': 1, 'class_names': ['barrier']},
  {'num_class': 2, 'class_names': ['motorcycle', 'bicycle']},
  {'num_class': 2, 'class_names': ['pedestrian', 'traffic_cone']}],
 'common_heads': {'reg': (2, 2),
  'height': (1, 2),
  'dim': (3, 2),
  'rot': (2, 2),
  'vel': (2, 2)},
 'bbox_coder': {'type': 'CenterPointBBoxCoder',
  'post_center_range': [-61.2, -61.2, -10.0, 61.2, 61.2, 10.0],
  'max_num': 500,
  'score_threshold': 0.1,
  'out_size

In [1]:
dict(type='SeparateHead',
                           init_bias=-2.19,
                           final_kernel=3)

{'type': 'SeparateHead', 'init_bias': -2.19, 'final_kernel': 3}